# 朝倉書店「医学への統計学」7章 頻度に関する推測

例題をJuliaを使って解いてみる。

In [56]:
using Distributions
using HypothesisTests

## 例題7.1

ある疾患の罹患状況を調査する目的で、100人を1年間追跡する計画を立てた。その結果、4人は、それぞれ、0.4、0.5、0.7、0.9年で罹患し、2人は、それぞれ、0.2、0.8年追跡したところで追跡不能（lost to follow-up）となり、残りの94人は1年間罹患しなかった。この場合の人年と罹患率を求めよ。

In [14]:
person_years = (0.4+0.5+0.7+0.9) + (0.2+0.8) + 1.0*94
incidence_case = 4
incidence_rate = (incidence_case / person_years) * 1000
println("人年: ", person_years)
println("罹患率（人/1000人年）: ", round(incidence_rate, digits=2))

人年: 97.5
罹患率（人/1000人年）: 41.03


## 例題7.2

大気汚染の汚染地区に指定されているA地区の住民676名を無作為に抽出し、94%の635名について回答を得た（回答率94%）。その中で、’持続性せき’の症状を持っていた者が68名いた。’持続性せき’の全国平均の割合は8.2%であるが、この地区の割合は全国平均と比べて高いと言えるか。また、この地区の有症率の95%信頼区間を求めよ。

In [38]:
function calc_z_stat(n, p_0, p_hat, α)
    nume = abs(p_hat - p_0) - 1/(2*n)
    denom = √((p_0*(1-p_0))/n)
    z_stat = nume / denom
    p_point = cquantile(Normal(0, 1), α/2)
    return z_stat, p_point
end


function calc_confidence_intervals(n, p_hat, p_point)
    lower = p_hat - p_point*√((p_hat*(1-p_hat))/n)
    upper = p_hat + p_point*√((p_hat*(1-p_hat))/n)
    return lower, upper
end

calc_confidence_intervals (generic function with 1 method)

In [40]:
r = 68
n = 635
p_0 = 0.082
p_hat = r / n
α = 0.05

z_stat, p_point = calc_z_stat(n, p_0, p_hat, α)
lower, upper = calc_confidence_intervals(n, p_hat, p_point)

println("統計量: ", round(z_stat, digits=3))
println("上側パーセント点(α=0.05): ", round(p_point, digits=3))
println(round(lower, digits=3), " ≤ p ≤ ", round(upper, digits=3))

統計量: 2.232
上側パーセント点(α=0.05): 1.96
0.083 ≤ p ≤ 0.131


従って、有意水準5%で全国平均より高く、95%信頼区間は8.3〜13.1%となる。

## 例題7.3

狭心症に対して有効な交感神経$\beta$ブロッカーとして使われているA薬の治療効果はだいたい65%であることが知られている。今度、新しく開発されたB薬の治療効果について試験を行ったところ、43例中31名に効果が認められるという結果を得た。B薬は旧来のA薬に比べて優れていると判定できるか。また、B薬による治療効果の割合の95%信頼区間を求めよ。

In [41]:
r = 31
n = 43
p_0 = 0.65
p_hat = r / n
α = 0.05

z_stat, p_point = calc_z_stat(n, p_0, p_hat, α)
lower, upper = calc_confidence_intervals(n, p_hat, p_point)

println("統計量: ", round(z_stat, digits=3))
println("上側パーセント点(α=0.05): ", round(p_point, digits=3))
println(round(lower, digits=3), " ≤ p ≤ ", round(upper, digits=3))

統計量: 0.815
上側パーセント点(α=0.05): 1.96
0.587 ≤ p ≤ 0.855


従って、有意水準5%で旧来のA薬よりB薬の方が優れているとは認められない。B薬の治療効果が期待できる割合の95%信頼区間は58.7〜85.5%となる。

## 例題7.4

心筋梗塞と食品抗体に関する研究で、牛乳蛋白に対する抗体の有無によって、心筋梗塞発作後6カ月以内に死亡する割合が変化するかどうかを213名の心筋梗塞患者について調べた。牛乳抗体の陽性・陰性により発作後6カ月以内の死亡の割合に差があるか、また差の95%信頼区間を求めよ。

In [52]:
function calc_z_stat(n₁, n₂, r₁, r₂, α)
    p̂₁ = r₁ / n₁
    p̂₂ = r₂ / n₂
    p̄ = (r₁+r₂) / (n₁+n₂)
    q̄ = 1 - p̄
    nume = abs(p̂₁-p̂₂) - 0.5*(1/n₁ + 1/n₂)
    denom = √(p̄*q̄ * (1/n₁ + 1/n₂))
    z_stat = nume / denom
    p_point = cquantile(Normal(0, 1), α/2)
    return p̂₁, p̂₂, z_stat, p_point
end

calc_z_stat (generic function with 3 methods)

In [53]:
function calc_confidence_intervals(n₁, n₂, p̂₁, p̂₂, p_point)
    lower = p̂₁ - p̂₂ - p_point * √((p̂₁*(1-p̂₁))/n₁ +(p̂₂*(1-p̂₂))/n₂)
    upper = p̂₁ - p̂₂ + p_point * √((p̂₁*(1-p̂₁))/n₁ +(p̂₂*(1-p̂₂))/n₂)
    return lower, upper
end

calc_confidence_intervals (generic function with 2 methods)

In [54]:
n₁ = 109
n₂ = 104
r₁ = 29
r₂ = 10
α = 0.05

p̂₁, p̂₂, z_stat, p_point = calc_z_stat(n₁, n₂, r₁, r₂, α)
lower, upper = calc_confidence_intervals(n₁, n₂, p̂₁, p̂₂, p_point)

println("牛乳抗体陽性の場合の6カ月以内の死亡率: ", round(p̂₁, digits=3))
println("牛乳抗体陰性の場合の6カ月以内の死亡率: ", round(p̂₂, digits=3))
println("統計量: ", round(z_stat, digits=3))
println("上側パーセント点(α=$α): ", round(p_point, digits=3))
println(round(lower, digits=3), " ≤ p₁-p₂ ≤ ", round(upper, digits=3))

牛乳抗体陽性の場合の6カ月以内の死亡率: 0.266
牛乳抗体陰性の場合の6カ月以内の死亡率: 0.096
統計量: 3.028
上側パーセント点(α=0.05): 1.96
0.069 ≤ p₁-p₂ ≤ 0.27


従って、有意水準5%で牛乳抗体陽性の方が発作後6カ月以内の死亡の割合が高く、その差の95%信頼区間は0.069〜0.27となる。

## 例題7.5

例題7.4をカイ二乗検定で行え。

In [63]:
function calc_chi_square_stat(a, b, c, d, α)
    n1 = a + b
    n2 = c + d
    N = n1 + n2
    m1 = a + c
    m2 = b + d
    nume = N * (abs(a*d - b*c) - N/2)^2
    denom = n1 * n2 * m1 *m2 
    chi_square_stat = nume / denom
    p_point = cquantile(Chisq(1), α)
    return chi_square_stat, p_point
end

calc_chi_square_stat (generic function with 2 methods)

In [65]:
a = 29; b = 80
c = 10; d = 94
α = 0.05
chi_square_stat, p_point = calc_chi_square_stat(a, b, c, d, α)
println("統計量: ", round(chi_square_stat, digits=3))
println("上側パーセント点(α=$α): ", round(p_point, digits=3))

統計量: 9.167
上側パーセント点(α=0.05): 3.841


従って、有意水準5%でグループ間に差があることが認められた。

## 例題7.6

ある新薬の初期の臨床試験で、新薬20例、対照薬10例に割り当てた。その結果、副作用の有無の結果を得た（それぞれ1例ずつ不採用）。副作用の発生に差があるか。

In [108]:
n = [19, 9]
m = [5, 23]
N = sum(n)
p = ones(length(as))
as = 0:5

for a in as
    b = n[1] - a
    c = m[1] - a
    d = n[2] - c
    p[a+1] = ((binomial(m[1],a)*binomial(m[2],b)))/binomial(N,n[1])
end

hcat(as, p)

6×2 Matrix{Float64}:
 0.0  0.00128205
 1.0  0.024359
 2.0  0.146154
 3.0  0.354945
 4.0  0.354945
 5.0  0.118315

In [109]:
正確な片側p値 = p[1] + p[2]
正確な両側p値 = 正確な片側p値 * 2
println("正確な片側p値: ", round(正確な片側p値, digits=3))
println("正確な両側p値: ", round(正確な両側p値, digits=3))

正確な片側p値: 0.026
正確な両側p値: 0.051


## 例題7.8

アセプトロールの狭心症に対する治療効果について二重盲検法でクロスオーバー法を用いて検討した。対照薬としてはアルプリノロールを用い、患者は54名を対象とした。改善度に有意差が認められるか。